# Installation

In [15]:
!pip install numpy
!pip install scipy
!pip install chromadb -q
!pip install xformers -q
!pip install langchain -q
!pip install flash_attn -q
!pip install langchain-community -q
!pip install sentence-transformers -q
!pip install langchain-huggingface -q
!pip install chromadb
!pip install langchain_nvidia_ai_endpoints
!pip install langgraph
!pip install gradio
!pip install PyPDF2 -q


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.3/15.3 MB 80.5 MB/s eta 0:00:00:00:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.0/6.0 MB 50.8 MB/s eta 0:00:0000:0100:01
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 29.9 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 45.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 412.7/412.7 kB 22.8 MB/s eta 0:00:00


In [16]:
from langchain_huggingface import HuggingFaceEmbeddings
from tqdm.autonotebook import tqdm, trange
from langchain_core.documents import Document
import chromadb
from langchain_community.vectorstores import Chroma
from langchain import hub
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_nvidia_ai_endpoints import ChatNVIDIA
from langchain_core.prompts import PromptTemplate
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.pydantic_v1 import BaseModel, Field
from pprint import pprint
from langgraph.graph import END, StateGraph, START
from typing import List
from langchain_core.documents import Document
from langchain_community.tools.tavily_search import TavilySearchResults
from typing_extensions import TypedDict
import pandas as pd
import torch
import os
import io
from bs4 import BeautifulSoup

<ipython-input-16-c0feb6342847>:2: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange


In [22]:
os.environ["TAVILY_API_KEY"] = "YOUR TAVILY_API_KEY"
os.environ["NVIDIA_API_KEY"] ="YOUR NVIDIA_API_KEY"
os.environ["LANGCHAIN_PROJECT"] = "RAG project"

# Eembedding_model

In [17]:
model_name = "dunzhang/stella_en_1.5B_v5"

model_kwargs = {
                'trust_remote_code': 'True'
                }
embedding_model = HuggingFaceEmbeddings(model_name=model_name, show_progress=True, model_kwargs=model_kwargs)


modules.json:   0%|          | 0.00/316 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/397 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/169k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/51.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/844 [00:00<?, ?B/s]

...

special_tokens_map.json:   0%|          | 0.00/370 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/289 [00:00<?, ?B/s]

2_Dense_1024/config.json:   0%|          | 0.00/122 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/6.30M [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/6.30M [00:00<?, ?B/s]

# Load your tool documentation.

In [6]:
dataiku_folder = "/kaggle/working/flowise_docs"

d = {"chunk": [], "url": []}

for path in os.listdir(dataiku_folder):
    url = "https://" + path.replace("=", "/")
    file_path = os.path.join(dataiku_folder, path)
    
    with open(file_path, 'rb') as stream:
        content = stream.read().decode("utf-8")
        
        # Parse the HTML content
        soup = BeautifulSoup(content, "html.parser")
        
        # Extract the title
        title = soup.find("title")
        title_text = title.string.replace(" | Dataiku", "") if title else "No Title"
        
        # Extract the main content (you might need to adjust this selector)
        main_content = soup.find("main")
        if main_content:
            text_content = main_content.get_text(strip=True)
        else:
            text_content = soup.get_text(strip=True)
        
        # Combine title and content
        full_content = f"{title_text}\n\n{text_content}"
        
        # Add to our dictionary
        d["chunk"].append(full_content)
        d["url"].append(url)

# Create a DataFrame from the dictionary
df = pd.DataFrame(d)

# Display the first few rows of the DataFrame
print(df.head())

# Optionally, save to a CSV file
# df.to_csv("dataiku_docs_chunks.csv", index=False)

                                               chunk  \
0  \n   Read File | FlowiseAI\n  \n\nIntegrations...   
1  \n   Introduction | FlowiseAI\n  \n\nIntroduct...   
2  \n   Embed | FlowiseAI\n  \n\nUsing FlowiseEmb...   
3  \n   VectorStore To Document | FlowiseAI\n  \n...   
4  \n   Introduction | FlowiseAI\n  \n\nIntroduct...   

                                                 url  
0  https://integrations_langchain_tools_read-file...  
1                      https://#retriever-nodes.html  
2     https://using-flowise_embed?fallback/true.html  
3  https://integrations_langchain_document-loader...  
4  https://#database.tf-would-define-the-configur...  


In [7]:
df["chunk_id"] = range(len(df))

In [8]:
list_of_documents = []

# Converts the dataframe into list of dictionaries each dictionary has 'chunk' as key and its 'url' as values i.e. records
records = df[['chunk', 'url']].to_dict(orient='records')

for record in records:
    list_of_documents.append(
        Document(
            page_content = record['chunk'],
            metadata={"source_url": record['url']}
        )
    )

In [9]:
print(list_of_documents[:5][0].page_content)


   Read File | FlowiseAI
  

IntegrationsLangChainToolsRead FileRead file from disk.PreviousCode Interpreter by E2BNextRequest GetLast updated6 days ago


In [10]:
df[['chunk', 'url']].to_dict(orient='records')[:2]


[{'chunk': '\n   Read File | FlowiseAI\n  \n\nIntegrationsLangChainToolsRead FileRead file from disk.PreviousCode Interpreter by E2BNextRequest GetLast updated6 days ago',
  'url': 'https://integrations_langchain_tools_read-file?fallback/true.html'},
 {'chunk': '\n   Introduction | FlowiseAI\n  \n\nIntroductionWelcome to the official Flowise documentationNextGet StartedLast updated6 days ago',
  'url': 'https://#retriever-nodes.html'}]

In [11]:
# Convert ids to list of strings of ids for Chromadb 
ids = [str(i) for i in df['chunk_id'].to_list()]

In [37]:
# Initiate persistent Chroma DB client to work around the max_batch_size limit
client = chromadb.PersistentClient()
collection = client.get_or_create_collection("rag-chroma")

vector_store = Chroma(
    client=client,
    collection_name="rag-chroma",
    embedding_function=embedding_model,
)

In [38]:
# Use the same loop for adding documents from the list of documents

start_index = 0
max_batch_size = 5461
total_len = len(list_of_documents)

for i in range(1, total_len//5461 + 2):
    
    end_index = i*5461

    if 54500 - start_index < 5461:
        vector_store.add_documents(documents=list_of_documents[start_index:], ids=ids)
        break
        
    else:
        vector_store.add_documents(documents=list_of_documents[start_index:end_index], ids=ids)
        
    start_index = end_index

Batches:   0%|          | 0/46 [00:00<?, ?it/s]

In [39]:
retriever = vector_store.as_retriever(search_kwargs={"k": 7})
retriever.invoke("FlowiseAI supports integration with popular large language models")

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

[Document(metadata={'source_url': 'https://integrations?fallback/true.html'}, page_content='\n   Integrations | FlowiseAI\n  \n\nIntegrationsLearn about all available integrations / nodes in FlowiseIn Flowise, nodes are referred to as integrations. Similar to LEGO, you can build a customized LLM ochestration flow, a chatbot, an agent with all the integrations available in Flowise.LangChainAgentsCacheChainsChat ModelsDocument LoadersEmbeddingsLLMsMemoryModerationOutput ParsersPromptsRecord ManagersRetrieversText SplittersToolsVector StoresLlamaIndexAgentsChat ModelsEmbeddingsEngineResponse SynthesizerToolsVector StoresUtilitiesCustom JS FunctionSet/Get VariableIf ElseSet VariableSticky NoteExternal IntegrationsZapier ZapsPreviousRunning in ProductionNextLangChainLast updated6 days ago'),
 Document(metadata={'source_url': 'https://integrations_langchain_chat-models?fallback/true.html'}, page_content='\n   Chat Models | FlowiseAI\n  \n\nIntegrationsLangChainChat ModelsLangChain Chat Model

In [40]:
retriever=vector_store.as_retriever()

# Retrieval grader

In [41]:
# Prompt
prompt = PromptTemplate.from_template(
"""
You are an assistant for responding to Request For Proposal documents for a 
bidder in the field of Data Science and Engineering. 
You will be given technical requirements point. 
Use the following pieces of retrieved context to respond to the requests. 
If you don't know the answer, just say that you don't know. 
Analyze the request carefully and make sure that your answer 
addresses the topic of the request. Base your answer on the most 
relevant piece of context.

Question: {question} 
Context: {context} 
Answer:
"""
)


# LLM
llm = ChatNVIDIA(model="meta/llama-3.3-70b-instruct", temperature=0)

# Post-processing
def format_docs(result):
    return "\n\n".join(doc.page_content for doc in result)

# Chain
rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

# Run
# generation = rag_chain.invoke(question)
# print(generation)

In [42]:
### Retrieval grader

# Data model
class GradeDocuments(BaseModel):
    """Binary score for relevance check on retrieved documents."""

    binary_score: str = Field(
        description="Documents are relevant to the question, 'yes' or 'no'"
    )


# LLM
llm = ChatNVIDIA(model="meta/llama-3.3-70b-instruct", temperature=0)

# Structured output
structured_llm_grader = llm.with_structured_output(GradeDocuments)

# Prompt
system = """You are a grader assessing relevance of a retrieved document to a client request. \n 
    If the document contains keyword(s) or semantic meaning related to the client request, grade it as relevant. \n
    It does not need to be a stringent test. The goal is to filter out erroneous retrievals. \n
    Analyze the request and the documents and decide whether FlowiseAI complies with the request based on the retrieved documents. \n
    Give a binary score 'yes' or 'no' score to indicate whether the document is relevant to the question."""
grade_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system),
        ("human", "Retrieved document: \n\n {document} \n\n User question: {question}"),
    ]
)

retrieval_grader = grade_prompt | structured_llm_grader


# Web Search Tool

In [43]:
web_search_tool = TavilySearchResults(k=3)

In [44]:
# Web search
docs = web_search_tool.invoke({"query": "FlowiseAI on premise installation"})
web_results = "\n".join([d["content"] for d in docs])
web_results = Document(page_content=web_results)
print(web_results)

page_content='Follow our installation guide to start using FlowiseAI. 1. Login or create an account. To access Elestio, you will need to connect to your existing account or create a new one. We provide 4 ways to connect to your account: Github; Gitlab; Google; Email; MFA is enable by default for new accounts created with an email address.
Flowise AI Modules. Flowise AI breaks down into three core modules: Server: Handles data processing, operations, task execution UI: The graphical interface for interacting with Flowise AI Components: Individual building blocks that provide functionality. Together these modules create the Flowise AI ecosystem. But each can also be tweaked or
It is free to deploy the app on Huggingface. Flowise is not available as a one-click app on Coolify, but you can install it with Dockerfile. DATABASE_PATH=/app/storage APIKEY_PATH=/app/storage LOG_PATH=/app/storage SECRETKEY_PATH=/app/storage Step3: Set the Persistent Storage (destination path) as /app/storage Flow

# Langgraph

In [45]:
class GraphState(TypedDict):
    """
    Represents the state of our graph.

    Attributes:
        question: question
        generation: LLM generation
        documents: list of documents
    """

    question: str
    generation: str
    decision: str
    documents: List[str]



In [46]:
# Nodes langgraph (python functions)

def retrieve(state):
    """
    Retrieve documents

    Args:
        state (dict): The current graph state

    Returns:
        state (dict): New key added to state, documents, that contains retrieved documents
    """
    print("---RETRIEVE---")
    question = state["question"]

    # Retrieval
    documents = retriever.invoke(question)
    
    return {"documents": documents, "question": question}


def retrieval_grader_function(state):


    print("---CHECK DOCUMENT RELEVANCE TO QUESTION---")
    question = state["question"]
    documents = state["documents"]

    # Score each doc
    filtered_docs = []
    for d in documents:
        score = retrieval_grader.invoke(
            {"question": question, "document": d.page_content}
        )
        grade = score.binary_score
        if grade == "yes":
            # print("---GRADE: DOCUMENT RELEVANT---")
            filtered_docs.append(d)
        else:
            # print("---GRADE: DOCUMENT NOT RELEVANT---")
            continue


    if len(filtered_docs) == 0:

        print("---NO RELEVANT DOCUMENTS FOUND---")

        print("\nNo relevant documents were found. going to websearch ...")

        print("\nDecision is Not Compliant ...")

        return {"question": question, "decision": "Partially Compliant or Not Compliant"}
    
    else:
        print("---RELEVANT DOCUMENTS FILTERED---")
        print("decision:", len(filtered_docs), "Relevant documents found and will be used to answer")

        return {"documents": filtered_docs, "question": question, "decision": "Fully Compliant"}

    

def compliance(state):

    print("---CHECK SOURCE---")
    question = state["question"]
    # documents = state["documents"]
    # generation = state["generation"]

    
    comp = question_router.invoke({"client_request": question})

    compliance_decision = comp.ComplianceType



    return {"decision": compliance_decision, "question": question}


def condition(state):

    dec = state["decision"]

    print(dec)

    if dec == "Fully Compliant":
        print("Decision: The request is fully compliant and local document will be used: \n \n")

        return "FC"
    
    # elif dec == "Partially Compliant or Not Compliant":
    #     print("Decision: The request is partially compliant or not compliant and web search will be used.")

    #     return "PC_or_NC"
    
    else:
        print("Decision: The request is partially compliant or not compliant.")


        return "PC or NC"




def generate_rag(state):

    print("I should be running only when the decision is Full Compliant")

    question = state["question"]
    decision = state["decision"]
    
    generation = rag_chain.invoke(question)

    return {"generation": generation}


def web_search(state):
    """
    Web search based on the re-phrased question.

    Args:
        state (dict): The current graph state

    Returns:
        state (dict): Updates documents key with appended web results
    """

    print("---WEB SEARCH---")
    question = state["question"]

    # Web search
    docs = web_search_tool.invoke({"query": question})
    web_results = "\n".join([d["content"] for d in docs])
    web_results = Document(page_content=web_results)

    return {"documents": web_results, "question": question}

In [47]:
workflow = StateGraph(GraphState)

# Define the nodes
workflow.add_node("Retrieve", retrieve)
workflow.add_node("Retrieval grader", retrieval_grader_function)
workflow.add_node("generate_rag", generate_rag)  # use RAG
workflow.add_node("web_search", web_search)

# Build graph
workflow.add_edge(START, "Retrieve")
workflow.add_edge("Retrieve", "Retrieval grader")
workflow.add_conditional_edges(
    "Retrieval grader",
    condition,
    {
        "FC": "generate_rag",
        "PC or NC": "web_search"
    },
    
)

workflow.add_edge("web_search", "generate_rag")
workflow.add_edge("generate_rag", END)



# Compile
app = workflow.compile()


# Load the requirements file.

In [48]:
cm=pd.read_csv("/kaggle/input/requirement/requirements.csv",encoding='latin-1')

# Run the Graph

In [49]:
for request in cm:

    # Run
    inputs = {
        "question": request
    }

    for output in app.stream(inputs):
        for key, value in output.items():
            # Node
            pprint(f"Node '{key}':")
            # Optional: print full state at each node
            # pprint.pprint(value["keys"], indent=2, width=80, depth=None)
        pprint("\n---\n")

    
    # Final generation
    pprint(value["generation"])
    

---RETRIEVE---


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

"Node 'Retrieve':"
'\n---\n'
---CHECK DOCUMENT RELEVANCE TO QUESTION---
---RELEVANT DOCUMENTS FILTERED---
decision: 2 Relevant documents found and will be used to answer
Fully Compliant
Decision: The request is fully compliant and local document will be used: 
 

"Node 'Retrieval grader':"
'\n---\n'
I should be running only when the decision is Full Compliant


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

"Node 'generate_rag':"
'\n---\n'
('The solution provides capabilities to connect to NoSQL databases, '
 'specifically MongoDB Atlas, a managed cloud MongoDB database. This is '
 'evident from the context, which mentions "Upsert embedded data and perform '
 'similarity or mmr search upon query using MongoDB Atlas". This indicates '
 'that the solution has the ability to connect to MongoDB Atlas, a NoSQL '
 'database, and perform various operations on it. \n'
 '\n'
 "However, it's worth noting that the context also mentions support for other "
 'database types, including SQLite, MySQL, PostgreSQL, and MariaDB, but these '
 'are primarily relational databases, not NoSQL databases. Therefore, the '
 "solution's capability to connect to NoSQL databases is specifically "
 'highlighted through its support for MongoDB Atlas.')
---RETRIEVE---


...

"Node 'generate_rag':"
'\n---\n'
("I don't know about CouchDB as it is not mentioned in the provided context. "
 'The context only mentions other databases and technologies such as '
 'SingleStore, Elastic, Redis, Supabase, and SQL Database Chain, but does not '
 'provide any information about CouchDB.')
---RETRIEVE---


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

"Node 'Retrieve':"
'\n---\n'
---CHECK DOCUMENT RELEVANCE TO QUESTION---
---RELEVANT DOCUMENTS FILTERED---
decision: 4 Relevant documents found and will be used to answer
Fully Compliant
Decision: The request is fully compliant and local document will be used: 
 

"Node 'Retrieval grader':"
'\n---\n'
I should be running only when the decision is Full Compliant


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

"Node 'generate_rag':"
'\n---\n'
('It seems like there is no actual question provided, just a series of context '
 'snippets related to FlowiseAI and Elastic. Could you please provide the '
 "actual question or technical requirement you'd like me to respond to? I'll "
 'do my best to address it based on the given context or let you know if I '
 "don't have enough information to provide an answer.")
